In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import precision_score, recall_score, f1_score

img_height, img_width = 228, 228
train_transforms = transforms.Compose([
    transforms.Resize((img_height, img_width)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
test_transforms = transforms.Compose([
    transforms.Resize((img_height, img_width)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

NUM_WORKERS = os.cpu_count()

def create_dataloaders(train_dir, test_dir, train_transform, test_transform, batch_size, num_workers=NUM_WORKERS):
    train_data = datasets.ImageFolder(train_dir, transform=train_transform)
    test_data = datasets.ImageFolder(test_dir, transform=test_transform)
    class_names = train_data.classes

    train_dataloader = DataLoader(
        train_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True
    )
    test_dataloader = DataLoader(
        test_data,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True
    )

    return train_dataloader, test_dataloader, class_names

# Setup dataloaders
train_dir = r'F:\ML_Projects\Image_Classification\mohan1\train'
test_dir = r'F:\ML_Projects\Image_Classification\mohan1\test'

train_dataloader, test_dataloader, class_names = create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    train_transform=train_transforms,
    test_transform=test_transforms,
    batch_size=32
)

model = models.vgg19(pretrained=True)
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, len(class_names))

for param in model.features[:-5].parameters():
    param.requires_grad = False

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)  # Learning rate scheduler

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    scheduler.step()  # Update learning rate

    model.eval()
    correct = 0
    total = 0
    predicted_labels = []
    true_labels = []
    with torch.no_grad():
        for inputs, labels in test_dataloader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            predicted_labels.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    accuracy = correct / total
    precision = precision_score(true_labels, predicted_labels, average='weighted')
    recall = recall_score(true_labels, predicted_labels, average='weighted')
    f1 = f1_score(true_labels, predicted_labels, average='weighted')

    epoch_loss = running_loss / len(train_dataloader.dataset)

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")


E:\Anaconda\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
E:\Anaconda\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/20], Loss: 1.8074, Accuracy: 0.5213, Precision: 0.5700, Recall: 0.5213, F1 Score: 0.4797
Epoch [2/20], Loss: 1.4969, Accuracy: 0.6429, Precision: 0.6492, Recall: 0.6429, F1 Score: 0.6410
Epoch [3/20], Loss: 1.2463, Accuracy: 0.6231, Precision: 0.6781, Recall: 0.6231, F1 Score: 0.6233
Epoch [4/20], Loss: 1.0475, Accuracy: 0.6960, Precision: 0.7110, Recall: 0.6960, F1 Score: 0.6930
Epoch [5/20], Loss: 0.9008, Accuracy: 0.7447, Precision: 0.7483, Recall: 0.7447, F1 Score: 0.7404
Epoch [6/20], Loss: 0.7140, Accuracy: 0.7568, Precision: 0.7702, Recall: 0.7568, F1 Score: 0.7569
Epoch [7/20], Loss: 0.5934, Accuracy: 0.6839, Precision: 0.7115, Recall: 0.6839, F1 Score: 0.6795
Epoch [8/20], Loss: 0.4797, Accuracy: 0.7599, Precision: 0.7688, Recall: 0.7599, F1 Score: 0.7608
Epoch [9/20], Loss: 0.3485, Accuracy: 0.7720, Precision: 0.7771, Recall: 0.7720, F1 Score: 0.7732
Epoch [10/20], Loss: 0.3173, Accuracy: 0.7842, Precision: 0.7934, Recall: 0.7842, F1 Score: 0.7860
Epoch [11/20], Loss

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import precision_score, recall_score, f1_score

img_height, img_width = 224, 224
train_transforms = transforms.Compose([
    transforms.Resize((img_height, img_width)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
test_transforms = transforms.Compose([
    transforms.Resize((img_height, img_width)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

NUM_WORKERS = os.cpu_count()

def create_dataloaders(train_dir, test_dir, train_transform, test_transform, batch_size, num_workers=NUM_WORKERS):
    train_data = datasets.ImageFolder(train_dir, transform=train_transform)
    test_data = datasets.ImageFolder(test_dir, transform=test_transform)
    class_names = train_data.classes

    train_dataloader = DataLoader(
        train_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True
    )
    test_dataloader = DataLoader(
        test_data,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True
    )

    return train_dataloader, test_dataloader, class_names

train_dir=r'F:\ML_Projects\Image_Classification\mohan1\train'
test_dir =  r'F:\ML_Projects\Image_Classification\mohan1\test'

train_dataloader, test_dataloader, class_names = create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    train_transform=train_transforms,
    test_transform=test_transforms,
    batch_size=32
)

model = models.vgg19(pretrained=True)
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, len(class_names))

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Store loss values and confusion matrix
train_losses = []
test_losses = []
all_predicted_labels = []
all_true_labels = []

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_train_loss = 0.0
    for inputs, labels in train_dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_train_loss += loss.item()

    train_losses.append(running_train_loss / len(train_dataloader))

    model.eval()
    running_test_loss = 0.0
    correct = 0
    total = 0
    predicted_labels = []
    true_labels = []
    with torch.no_grad():
        for inputs, labels in test_dataloader:

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            predicted_labels.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    test_losses.append(running_test_loss / len(test_dataloader))
    all_predicted_labels.extend(predicted_labels)
    all_true_labels.extend(true_labels)

    accuracy = correct / total
    precision = precision_score(true_labels, predicted_labels, average='weighted')
    recall = recall_score(true_labels, predicted_labels, average='weighted')
    f1 = f1_score(true_labels, predicted_labels, average='weighted')

    print(f"Epoch [{epoch+1}/{num_epochs}], Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}")

# Save loss values and confusion matrix
conf_matrix = confusion_matrix(all_true_labels, all_predicted_labels)
torch.save({
    'train_losses': train_losses,
    'test_losses': test_losses,
    'conf_matrix': conf_matrix,
    'class_names': class_names
}, 'metrics.pth')


In [ ]:
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def plot_loss_curves_and_confusion_matrix(metrics_file):
    # Load metrics
    metrics = torch.load(metrics_file)
    train_losses = metrics['train_losses']
    test_losses = metrics['test_losses']
    conf_matrix = metrics['conf_matrix']
    class_names = metrics['class_names']

    # Plot train and test loss curves
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, len(train_losses) + 1), train_losses, label='Train Loss')
    plt.plot(range(1, len(test_losses) + 1), test_losses, label='Test Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Train and Test Loss Curves')
    plt.legend()
    plt.show()

    # Plot confusion matrix
    plt.figure(figsize=(10, 8))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

# Example usage
plot_loss_curves_and_confusion_matrix('metrics.pth')


In [ ]:
import torch

torch.save(model.state_dict(), 'model3_inception_n.pth')